In [ ]:
!pip install boto3


In [ ]:
#where do the EXIF errors happen?
#where are there corrupt images?

import PIL
from pathlib import Path
from PIL import UnidentifiedImageError
import os
import boto3
from PIL import Image
import pandas as pd
import io
from io import BytesIO
import warnings
warnings.filterwarnings("error")

access_Key = "key"
secret_Access_Key = "secret"

s3 = boto3.resource('s3',aws_access_key_id=access_Key, aws_secret_access_key=secret_Access_Key)

valdf = pd.read_csv('/content/valdff.csv', index_col = 0, header = 0, engine='python', dtype = {'breed_mapping': "Int64"})


def image_from_s3(bucket, key):
    key = f'allimgs/{key}'
    bucket = s3.Bucket(bucket)
    image = bucket.Object(key)
    img_data = image.get().get('Body').read()
    try:
      Image.open(io.BytesIO(img_data))
      return 'Sucess'
    except Exception as e:
      print(f'Problem opening image from S3: {e}')
      print(key) 
      return 'Error'
    except UserWarning as e:
      print(f'Problem opening image from S3: {e}')
      print(key) 
      return 'Error'

listofpotentialdrops = []

for i in range(len(valdf)):
  entry = valdf['full_filename'][i]
  try:

    response = image_from_s3('dogshelterimgs', entry)

    if response == 'Error':
      print(f'Index of error: {i}.')
      listofpotentialdrops.append(entry)

  except Exception as e:

    print(f'Problem: {e}')
    print(f'Filename:{entry}. Index:{i}.')
    listofpotentialdrops.append(entry)

In [ ]:
import glob
import piexif

exifimgs = ['26570.jpg','71810.jpg','29807.jpg', '6414.jpg','26571.jpg','6391.jpg','29804.jpg']
nfiles = 0

for filename in exifimgs:
    nfiles = nfiles + 1
    filename = '/' + filename
    print("About to process file %d, which is %s." % (nfiles,filename))
    piexif.remove(filename)

About to process file 1, which is /26570.jpg.
About to process file 2, which is /71810.jpg.
About to process file 3, which is /29807.jpg.
About to process file 4, which is /6414.jpg.
About to process file 5, which is /26571.jpg.
About to process file 6, which is /6391.jpg.
About to process file 7, which is /29804.jpg.


In [ ]:
import boto3
from botocore.exceptions import NoCredentialsError

access_Key = "ket"
secret_Access_Key = "secret"

def upload_kaggle_photo(filename, bucket):

  actualfilepath = '/' + filename

  for name in listofpotentialdrops:
    if actualfilepath in name:
      
      key = 'allimgs/' +name
      print(key)
  

  print(actualfilepath)



  s3 = boto3.client('s3', aws_access_key_id= access_Key, aws_secret_access_key=secret_Access_Key)
  try:
    s3.upload_file(actualfilepath,bucket,key)
  except FileNotFoundError:
    print('no file at link')
  except NoCredentialsError:
    print('credential error')


In [ ]:
for path in exifimgs:
  upload_kaggle_photo(path,'dogshelterimgs')

In [ ]:
import pandas as pd

valdf = pd.read_csv('/content/valdfclean.csv', index_col = 0, header = 0, engine='python', dtype = {'breed_mapping': "Int64"})


In [ ]:
for i in range(len(valdf)):
  if valdf['full_filename'][i] == 'Australian-Shepherd/42086.jpg':
    print(valdf['full_filename'][i])
    print(i)
exifimgs = ['26570.jpg','71810.jpg','29807.jpg', '6414.jpg','26571.jpg','6391.jpg','29804.jpg']
nfiles = 0

In [ ]:
valdf = valdf.drop([11513])
valdf.reset_index(inplace=True, drop=True)
valdf.to_csv('valdfclean2.csv')
validation_df = valdf[valdf['breed_mapping'] == 11]
validation_df.reset_index(inplace=True, drop=True)

In [ ]:
import boto3


access_Key = "key"
secret_Access_Key = "secret"

s3 = boto3.resource('s3', aws_access_key_id= access_Key, aws_secret_access_key=secret_Access_Key)
bucket = s3.Bucket('dogshelterimgs')
bucket_files = [x.key for x in bucket.objects.all()]

In [ ]:
bucket_files2 = [filename for filename in bucket_files if "allimgs" in filename]

In [ ]:
bucket_files2 = ['dogshelterimgs/'+filename for filename in bucket_files2]

In [ ]:
import pandas as pd

valdf = pd.read_csv('/content/valdfclean2.csv', index_col = 0, header = 0, engine='python', dtype = {'breed_mapping': "Int64"})

In [ ]:
filenames = list(valdf['full_filename'].unique())

In [ ]:
filenames1 = ['dogshelterimgs/allimgs/' + filee for filee in filenames]

In [ ]:
for i in range(0,1):
  !rm -r /content/dogshelterimgs

In [ ]:
import piexif

def getridofexif(filename):
  fs.download(filename,filename)
  piexif.remove(filename)
  fs.put(filename,filename)
  print(filename)

In [ ]:
for filename in filenames1[8290:]:
  if '.png' not in filename:
    getridofexif(filename)

In [ ]:
import s3fs
fs = s3fs.S3FileSystem(anon=True)
allfiles = fs.ls('my-bucket')

for filename in bucket_files:
  with fs.open(filename, 'rb') as f:
      print(f.read())



In [ ]:
#VGG-Finetuned-Evaluation-2021-08-26-01-18-13-111

fs.download('sagemaker-us-west-1-727244991076/VGG-Weighted-Evaluation-2021-08-26-18-26-11-287/output/output.tar.gz','evalresultsweight.tar.gz')

[None]

In [ ]:
import tarfile
with tarfile.open('evalresultsweight.tar.gz') as tar:
        tar.extractall(f'resultsweight')
        tar.close()

In [ ]:
preds = pd.read_csv('/content/results/predictions.csv', header=0, index_col=0)

In [ ]:
preds2 = pd.read_csv('/content/resultsweight/predictions.csv', header=0, index_col=0)

In [ ]:
testdf = pd.read_csv('/content/testdfclean.csv', header=0, index_col=0)
lifespans = pd.read_csv('/content/lifespans.csv', header=0)


In [ ]:
for i in range(len(preds)):
  if preds['Exact_Age_MAE'][i] == 'nan':
    lifespan = lifespans[testdf['primaryBreed'][i]][0]
    preds.at[i,'Pred_Exact_Age'] = float(preds['Pred_Norm_Age'][i]) * lifespan
    preds.at[i,'Exact_Age_MAE'] = abs(preds['Pred_Exact_Age'][i] - preds['Exact_Age'][i])


In [ ]:
for i in range(len(preds2)):
  if preds2['Exact_Age_MAE'][i] == 'nan':
    lifespan = lifespans[testdf['primaryBreed'][i]][0]
    preds2.at[i,'Pred_Exact_Age'] = float(preds2['Pred_Norm_Age'][i]) * lifespan
    preds2.at[i,'Exact_Age_MAE'] = abs(preds2['Pred_Exact_Age'][i] - preds2['Exact_Age'][i])


In [ ]:
lifespans['Akbash'][0]

10.5

In [ ]:
for i in range(len(preds)):
  preds.at[i,'Pred_Exact_Age'] = preds['Pred_Exact_Age'][i].replace('[','').replace(']','')
  preds.at[i,'Pred_Norm_Age'] = preds['Pred_Norm_Age'][i].replace('[','').replace(']','')
  preds.at[i,'Exact_Age_MAE'] = preds['Exact_Age_MAE'][i].replace('[','').replace(']','')
  preds.at[i,'Norm_Age_MAE'] = preds['Norm_Age_MAE'][i].replace('[','').replace(']','')

In [ ]:
len(preds)

13888

In [ ]:
len(youngonly)

4186

In [ ]:
oldonly = preds[preds['Norm_Age']>.5]
oldonly['Norm_Age_MAE'] = pd.to_numeric(oldonly['Norm_Age_MAE'])
oldonly['Exact_Age_MAE'] = pd.to_numeric(oldonly['Exact_Age_MAE'])

oldonly2 = preds2[preds2['Norm_Age']>.5]
oldonly2['Norm_Age_MAE'] = pd.to_numeric(oldonly2['Norm_Age_MAE'])
oldonly2['Exact_Age_MAE'] = pd.to_numeric(oldonly2['Exact_Age_MAE'])


In [ ]:
youngonly = preds[preds['Norm_Age']<.5]
youngonly['Norm_Age_MAE'] = pd.to_numeric(youngonly['Norm_Age_MAE'])
youngonly['Exact_Age_MAE'] = pd.to_numeric(youngonly['Exact_Age_MAE'])


In [ ]:
oldonly[['Norm_Age_MAE','Exact_Age_MAE']].describe()

,Norm_Age_MAE,Exact_Age_MAE
count,2.070000e+03,2070.000000
mean,3.401649e-01,4.036209
std,2.163911e-01,2.590702
min,3.576279e-07,0.000006
25%,1.788376e-01,2.104098
50%,3.225435e-01,3.779666
75%,4.656190e-01,5.520180
max,1.568055e+00,19.332592


In [ ]:
youngonly[['Norm_Age_MAE','Exact_Age_MAE']].describe()

,Norm_Age_MAE,Exact_Age_MAE
count,11775.000000,11775.000000
mean,0.082688,1.002276
std,0.092437,1.129243
min,0.000003,0.000043
25%,0.014796,0.181819
50%,0.047124,0.572648
75%,0.121265,1.460489
max,0.707121,10.746359


In [ ]:
oldonly

,Filename,Exact_Age,Norm_Age,Pred_Exact_Age,Pred_Norm_Age,Exact_Age_MAE,Norm_Age_MAE
1,Shepherd/57272.jpg,6.0,0.545455,4.3268137,0.3933467,1.673186,0.152108
18,Shepherd/2110.jpg,6.0,0.545455,2.9359822,0.26690748,3.064018,0.278547
30,American-Pit-Bull-Terrier/1994.jpg,9.0,0.818182,2.0438635,0.18580577,6.956137,0.632376
32,American-Bulldog/73552.jpg,7.0,0.560000,6.2215586,0.49772468,0.778441,0.062275
33,Boxer/3764.jpg,6.0,0.576923,1.4386525,0.13833198,4.561348,0.438591
...,...,...,...,...,...,...,...
13859,Chihuahua/15067.jpg,13.0,1.000000,9.95804,0.7660031,3.041960,0.233997
13868,Mixed-Breed/9b53610ea-8.jpg,7.0,0.560000,0.667964,0.05343712,6.332036,0.506563
13878,Dachshund/62591.jpg,11.0,0.901639,10.1628275,0.83301866,0.837172,0.068621
13880,Mixed-Breed/9b53610ea-15.jpg,7.0,0.560000,0.54950744,0.04396059,6.450492,0.516039
